In [ ]:
# Install Java 21 (required for modern Minecraft versions)
!sudo apt-get update
!sudo apt-get install -y wget apt-transport-https
!wget -O - https://packages.adoptium.net/artifactory/api/gpg/key/public | sudo tee /usr/share/keyrings/adoptium.asc
!echo "deb [signed-by=/usr/share/keyrings/adoptium.asc] https://packages.adoptium.net/artifactory/deb $(awk -F= '/^VERSION_CODENAME/{print$2}' /etc/os-release) main" | sudo tee /etc/apt/sources.list.d/adoptium.list
!sudo apt-get update
!sudo apt-get install temurin-21-jdk -y

In [ ]:
# Direct link provided by the user
!wget https://piston-data.mojang.com/v1/objects/28a9d30423b1c50da661b95e754be9d5f0c871d4/server.jar -O server.jar

In [11]:
!echo "eula=true" > eula.txt

In [ ]:
# Install playit.gg for tunneling
!curl -SsL https://playit-cloud.github.io/ppa/key.gpg | sudo apt-key add -
!sudo curl -SsL -o /etc/apt/sources.list.d/playit-cloud.list https://playit-cloud.github.io/ppa/playit-cloud.list
!sudo apt update
!sudo apt install playit -y

In [66]:
# Start playit.gg in the background (it will print a claim URL)
!pkill agent
!playit > playit.log 2>&1 &

In [ ]:
# Extract the URL to claim your tunnel
!cat playit.log | grep -o 'https://playit.gg/claim/[^ ]*'

In [ ]:
!wget https://raw.githubusercontent.com/VTSTech/Minecraft.SNAPSHOT.Commands.Reborn/refs/heads/main/snapshot.cmd.by.VTSTech.py

In [ ]:
# Replace the JVM memory flags in the Python script
#!sed -i 's/-Xms[0-9]\+[MG]\b/-Xms2G/g' snapshot.cmd.by.VTSTech.py
#!sed -i 's/-Xmx[0-9]\+[MG]\b/-Xmx4G/g' snapshot.cmd.by.VTSTech.py
# Use sed to replace the Java command line in the Python script
!sed -i 's/"java -Xms[0-9]\+[A-Z] -Xmx[0-9]\+[A-Z] -jar server.jar nogui"/"java -Xms6G -Xmx10G -XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Djava.awt.headless=true -jar server.jar nogui"/g' snapshot.cmd.by.VTSTech.py
# Verify the change
!grep 'java_cmd' snapshot.cmd.by.VTSTech.py
#replace motd
!sed -i 's/^motd=.*/motd=G! Colab SNAPSHOT Server by VTSTech/' server.properties

In [ ]:
!cat snapshot.cmd.by.VTSTech.py

In [85]:
# Allocate RAM (adjust -Xmx4G based on Colab's available memory)
!python3 snapshot.cmd.by.VTSTech.py > server.log 2>&1 &

In [ ]:
!cat server.log

In [ ]:
from google.colab import drive
import os
import time
import shutil
def backup_server():
    try:
        drive.mount('/content/drive', force_remount=True)
        backup_dir = '/content/drive/MyDrive/Minecraft_Backups'
        os.makedirs(backup_dir, exist_ok=True)

        timestamp = time.strftime("%Y%m%d-%H%M%S")
        backup_files = [
            'world',
            'versions',
            'server.properties',
            'usercache.json',
            'ops.json',
            'homes.csv',
            'server.jar',
            'eula.txt',
            'warps.csv'
        ]

        # Debug: Show current directory contents
        print("Current directory contents:")
        !ls -l

        # Create temporary directory
        temp_dir = f'temp_backup_{timestamp}'
        os.makedirs(temp_dir, exist_ok=True)
        print(f"Created temp dir: {temp_dir}")

        # Copy files with verification
        copied_files = []
        for item in backup_files:
            try:
                src_path = os.path.abspath(item)
                dest_path = os.path.join(temp_dir, item)

                print(f"\nChecking: {item}")
                print(f"Source path: {src_path}")
                print(f"Exists? {os.path.exists(src_path)}")

                if os.path.exists(src_path):
                    if os.path.isdir(src_path):
                        print(f"Copying directory: {src_path} → {dest_path}")
                        shutil.copytree(src_path, dest_path)
                    else:
                        print(f"Copying file: {src_path} → {dest_path}")
                        shutil.copy(src_path, dest_path)
                    copied_files.append(item)
                else:
                    print(f"⚠️ {item} not found")
            except Exception as e:
                print(f"❌ Error copying {item}: {str(e)}")

        print(f"\nCopied {len(copied_files)} items:")
        print("\n".join(copied_files))

        # Verify temp dir contents
        print("\nTemp dir contents:")
        !ls -l "{temp_dir}"

        # Create archive
        backup_path = os.path.join(backup_dir, f'full_backup_{timestamp}')
        print(f"\nCreating archive at: {backup_path}.tar.gz")
        shutil.make_archive(backup_path, 'gztar', temp_dir)

        # Verify archive
        print("\nArchive contents:")
        !tar -ztvf "{backup_path}.tar.gz"

        # Cleanup
        shutil.rmtree(temp_dir)
        print(f"\n✅ Backup complete: {backup_path}.tar.gz")
        return True

    except Exception as e:
        print(f"❌ Backup failed: {str(e)}")
        return False
    finally:
        drive.flush_and_unmount()
backup_server()

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Find latest backup
backup_dir = '/content/drive/MyDrive/Minecraft_Backups'
backups = sorted([f for f in os.listdir(backup_dir) if f.startswith('full_backup_')], reverse=True)

if backups:
    latest_backup = f'{backup_dir}/{backups[0]}'
    # Extract backup
    !tar -xzf "{latest_backup}" -C /content/
    print(f"Restored world from: {latest_backup}")
else:
    print("No backups found in Drive")

In [81]:
!pkill -f snapshot.cmd.by.VTSTech.py
!pkill -f java
# Allocate RAM (adjust -Xmx4G based on Colab's available memory)
!python3 snapshot.cmd.by.VTSTech.py > server.log 2>&1 &